In [ ]:
bike_data = 0

cat_cols_df = bike_data.select_dtypes(include=['object', 'category'])
num_cols_df = bike_data.select_dtypes(include=['int', 'float'])
cat_cols = list(cat_cols_df.columns)
num_cols = list(num_cols_df.columns)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

In [ ]:
pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaling',MinMaxScaler())
])

In [ ]:
pipeline_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoding', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', pipeline_num, num_cols),
        ('cat', pipeline_cat, cat_cols)])

In [ ]:
# List of regression models to test
models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest', RandomForestRegressor()),
    ('SVR', SVR()),
    ('Decision Tree', DecisionTreeRegressor())
]

In [ ]:
# To store results
results = {}

# Define a custom scorer for RMSE
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Loop over models and evaluate
for name, model in models:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),  # Assuming preprocessor is already defined
        ('regressor', model)
    ])
    
    # Perform cross-validation
    scores = cross_val_score(pipeline, X, y, cv=5, scoring=rmse_scorer)  # Replace X, y with your data
    results[name] = np.mean(scores)

# Print results
for name, score in results.items():
    print(f"{name}: RMSE = {score:.2f}")